# Alexandria3k Exercise 1 (Basics)
- Documentation: https://dspinellis.github.io/alexandria3k
- Python API examples: https://dspinellis.github.io/alexandria3k/python-eg.html

## Hello World Example from the documentation
- link: https://github.com/dspinellis/alexandria3k/tree/main/examples/authors-by-decade
A3K uses command line interface (`Makefile`) to interact with the database.
```bash
#
# Minimal application of Alexandria3k
# Report the average number of work authors per decade
#

export MAINDB?=hello
export DEPENDENCIES=populate

include ../common/Makefile

# Populate database with a 0.02% sample of Crossref containers
populate: $(CROSSREF_DIR)
	$(TIME) $(A3K) --debug progress \
	  populate "$(MAINDB).db" crossref "$(CROSSREF_DIR)" \
	  --sample 'random.random() < 0.0002' \
	  --columns works.id \
	    works.published_year \
	    work_authors.work_id \
	  --row-selection 'works.published_year >= 1940'
	touch $@
```
Since I am not familiar with the command line interface, I will use the Python API to interact with the database.

## Python API Example
The basic example I tried in the previous week is to sample 10% of the Crossref data and select the data satisfying the condition `title like '%causal%' AND container_title like '%software%'`. 

```python
from alexandria3k.data_sources.crossref import Crossref
from random import random, seed

def run_a3k():
    # Randomly (but deterministically) sample 10% of the containers
    seed("test")
    crossref_instance = Crossref('April 2023 Public Data File from Crossref', lambda _name: random() < 0.1)

    for (year, doi, title, venue) in (
            crossref_instance.query(
                "SELECT published_year, DOI, title, container_title FROM works WHERE title like '%causal%' AND container_title like '%software%'")):
        print(f'{year}: "{title}", {venue} [{doi}]')
```

It returned the following output:
```bash
2022: "Structural causal models as boundary objects in AI system development", Proceedings of the 1st International Conference on AI Engineering: Software Engineering for AI [10.1145/3522664.3528615]
2021: "Improved Causal Models of Alzheimer's Disease", 2021 IEEE 45th Annual Computers, Software, and Applications Conference (COMPSAC) [10.1109/compsac51774.2021.00046]
2008: "Guidance for Efficiently Implementing Defect Causal Analysis", Anais do VII Simpósio Brasileiro de Qualidade de Software (SBQS 2008) [10.5753/sbqs.2008.15540]
...
```

## Conclusion
- Given the large database `April 2023 Public Data File from Crossref`, the Python API works as expected.
- The database includes journal articles and conference/workshop papers; `container_title` is the venue of the publication.
- The search query is slow.
- Can't we handle intermediate databases for faster search?

# Alexandria3k Exercise 2 (SQLite Database)

To address the last question from the previous exercise, I studied the SQLite database in Python.

## What is SQLite?
- SQLite is a C library that provides a lightweight disk-based database.
- It doesn't require a separate server process.
- Compared to Pandas DataFrames, SQLite databases are more efficient for large datasets.

| Feature       | SQLite                                    | Pandas Dataframe                                  |
|---------------|-------------------------------------------|---------------------------------------------------|
| Type          | Database                                  | In-memory data structure                          |
| Persistence   | Persistent                                | In-memory, not persistent                         |
| Data Access   | SQL queries                               | Python functions and libraries                    |
| Scalability   | Limited for large datasets                | Limited for large datasets (memory-intensive)     |
| Relationships | Supports complex data relationships       | Limited to single table or joins                  |
| Visualization | Requires integration with other libraries | Seamless integration with visualization libraries |

## Example: SQLite Database in A3K using Python APIs
We can create a SQLite database from the Crossref data and query it using the Python API.
Since the Crossref data is large, we will use a database with Journal Names.

In [1]:
from alexandria3k.data_sources.journal_names import JournalNames

instance = JournalNames(
    "http://ftp.crossref.org/titlelist/titleFile.csv"
)
# Populate the database (basically convert the CSV into a SQLite DB)
instance.populate("journal-names.db")

In [2]:
# Query the database: Print the randomly sampled 5 rows of the table
import sqlite3

conn = sqlite3.connect('journal-names.db')
c = conn.cursor()
c.execute('SELECT * FROM journal_names ORDER BY RANDOM() LIMIT 5')

for item in c.fetchall():
    print(item)
conn.close()

(89399, 'Patient satisfaction following primary total knee arthroplasty in Sulaimani city', '393562', '"EDORIUM Journals, Pvt., Ltd."', '', '', '', '10.5348/100014O03RF2019RA', '[]')
(91556, 'Pflegewissenschaft Hungen', '549217', 'hpsmedia Verlag GmbH', '', '', '', '10.3936/docid200700', '[]')
(39100, 'IIUM Journal of Orofacial and Health Sciences', '402573', 'IIUM Press', '', '27350584', '', '', '"(2020)[1]1,2(2021)[2]1,2(2022)[3]1,2(2023)[4]1,2"')
(94741, 'Prace Naukowe Akademii im Jana Długosza w Częstochowie Studia Neofilologiczne', '297064', 'Uniwersytet Humanistyczno-Przyrodniczy im. Jana Dlugosza w Czestochowie', '18974244', '', '', '10.16926/sn.', '(2015)[11](2016)[12](2017)[13](2018)[14]()[]')
(66165, 'Journal of Survey in Fisheries Sciences', '558379', 'Green Publication', '', '', '', '10.53555//sfs.v10i1S.2311', '(2023)[]()[]')


Here, the `journal-names.db` is a SQLite database stored in the disk. 

In [3]:
# Check the size of the database
import os
print(f"Size of the database: {os.path.getsize('journal-names.db') / (1024 * 1024):.2f} MB")

Size of the database: 51.47 MB


Therefore, we can create an intermediate SQLite database and query it whenever necessary.
For example, we can create a smaller database for the "software" journals and query it for their publishers.

In [4]:
# Load the `journal-names.db` database
conn = sqlite3.connect('journal-names.db')
c = conn.cursor()

# Query the database: Find the "software" journals
c.execute("SELECT * FROM journal_names WHERE title LIKE '%software%'")
for item in c.fetchall():
    print(item)

(980, 'ACM SIGSOFT Software Engineering Notes', '29779', 'Association for Computing Machinery', '01635948', '', '', '', '"(1976)[1]1,2(1977)[2]1,2,3,4,5(1978)[3]1,2,3,4,5(1979)[4]1,2,3,4(1980)[5]1,2,3,4(1981)[6]1,2,3,4,5(1982)[7]1,2,3,4,5(1983)[8]1,2,3,4,5(1984)[9]1,2,3,4,5(1985)[10]1,2,3,4,5(1986)[11]1,2,3,4,5(1987)[12]1,2,3,4(1988)[13]1,2,3,4,5(1989)[14]1,2,3,4,5,6,7,8(1990)[15]1,2,3,4,5,6(1991)[16]1,2,3,4,5(1992)[17]1,2,3,4,5(1993)[18]1,2,3,4,5(1994)[19]1,2,3,4,5(1995)[20]1,2,3,4,5,si(1996)[21]1,2,3,4,5,6(1997)[22]1,2,3,4,5,6(1998)[23]1,2,3,4,5,6(1999)[24]1,2,3,4,5,6(2000)[25]1,2,3,4,5,6(2001)[26]1,2,3,4,5,6(2002)[27]1,2,3,4,5,6(2003)[28]1,2,3,4,5,6(2004)[29]1,2,3,4,5,6(2005)[30]1,2,3,4,5,6(2006)[31]1,2,3,4,5,6(2007)[32]1,2,3,4,5,6(2008)[33]1,2,3,4,5,6(2009)[34]1,2,3,4,5,6(2010)[35]1,2,3,4,5,6(2011)[36]1,2,3,4,5,6(2012)[37]1,2,3,4,5,6(2013)[38]1,2,3,4,5,6(2014)[39]1,2,3,4,5,6(2015)[40]1,2,3,4,5,6(2016)[41]1,2,3,4,5(2017)[41]6[42]1,2,3(2018)[42]4[43]1,2,3(2019)[43]4[44]1,2,3,4(2020)[

In [5]:
# Create a new database `software-journals.db` from the existing database `journal-names.db` using the query
c.execute("ATTACH DATABASE 'software-journals.db' AS new_db")
c.execute("CREATE TABLE new_db.journal_names AS SELECT * FROM journal_names WHERE title LIKE '%software%'")
conn.commit()
conn.close()

In [6]:
# Check the size of the new database `software-journals.db`
print(f"Size of the new database: {os.path.getsize('software-journals.db') / (1024 * 1024):.2f} MB")

Size of the new database: 0.07 MB


In [7]:
# Query the new database: Print the unique journal names as a list
conn = sqlite3.connect('software-journals.db')
c = conn.cursor()
c.execute("SELECT DISTINCT title FROM journal_names")
software_journal_list = [item[0] for item in c.fetchall()]
print(f'Total: {len(software_journal_list)}')
print(software_journal_list)

Total: 141
['ACM SIGSOFT Software Engineering Notes', 'ACM Transactions on Mathematical Software', 'ACM Transactions on Software Engineering and Methodology', 'Advances in Engineering Software', 'Advances in Engineering Software (1978)', 'Advances in Engineering Software and Workstations', 'Advances in Software Engineering', 'American Journal of Software Engineering', 'American Journal of Software Engineering and Applications', 'American Journal of Systems and Software', 'Annals of Software Engineering', 'Automated Software Engineering', 'Biotech Software & Internet Report', 'Bonfring International Journal of Software Engineering and Soft Computing', 'Bulletin of the South Ural State University Series Computational Mathematics and Software Engineering', 'Bulletin of the South Ural State University Series Mathematical Modelling Programming and Computer Software', 'Computer Software and Media Applications', 'Computing and Software for Big Science', 'e-Informatica Software Engineering Jou

In [8]:
# Query the new database: Print the unique publishers as a list
c.execute("SELECT DISTINCT publisher FROM journal_names")
software_publisher_list = [item[0] for item in c.fetchall()]
print(f'Total: {len(software_publisher_list)}')
print(software_publisher_list)

Total: 96
['Association for Computing Machinery', 'Elsevier ', 'Hindawi Limited', '"Science and Education Publishing Co., Ltd."', 'Science Publishing Group', 'Springer-Verlag', 'Mary Ann Liebert', 'Bonfring', 'FSAEIHE South Ural State University (National Research University)', 'EnPress Publisher', 'De Gruyter Poland Sp. z o.o.', 'Politechnika Wroclawska Oficyna Wydawnicza', 'Institution of Engineering and Technology (IET)', 'GITO mbH Verlag', 'i-manager Publications', 'Institute of Electrical and Electronics Engineers', 'Lattice Science Publication (LSP)', 'Institut Riset dan Publikasi Indonesia', 'Universitas Bina Sarana Informatika', 'Santoso Academy Network', 'Publishing House Helvetica', 'Universidad La Salle Arequipa', 'International Free and Open Source Software Law Review', 'Advance Academic Publisher', 'Inderscience Enterprises Ltd.', 'Graphy Publications', 'Dorma Trading Est', 'Universiti Malaysia Pahang Publishing', 'IGI Global', 'International Journal of Soft Computing and 